In [1]:
pip install pyiceberg

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install pyiceberg[s3fs,sql-sqlite]

In [3]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


LEER DESDE MINIO

In [5]:
!pip install --upgrade pyiceberg

In [6]:
!pip install --upgrade pyarrow

  Using cached pyarrow-21.0.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
Using cached pyarrow-21.0.0-cp311-cp311-manylinux_2_28_x86_64.whl (42.8 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 15.0.2
    Uninstalling pyarrow-15.0.2:
      Successfully uninstalled pyarrow-15.0.2


In [1]:
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import pyarrow.fs as fs
import os
import io
import requests
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.fs as pafs
import dlt

In [2]:
import pyarrow.fs as fs

minio = fs.S3FileSystem(
    access_key="admin",
    secret_key="password",
    endpoint_override="http://minio:9000",
    scheme="http"
)


In [4]:
path = "warehouse/yellow_trip_20250911092850/yellow_trip_data/1757626130.6596549.6e23b36ace.parquet"

In [5]:
dataset = ds.dataset(path, filesystem=minio, format= "parquet")

In [6]:
table = dataset.to_table()
print(table.schema)

vendor_id: int32
tpep_pickup_datetime: timestamp[us]
tpep_dropoff_datetime: timestamp[us]
passenger_count: int64
trip_distance: double
ratecode_id: int64
store_and_fwd_flag: large_string
pu_location_id: int32
do_location_id: int32
payment_type: int64
fare_amount: double
extra: double
mta_tax: double
tip_amount: double
tolls_amount: double
improvement_surcharge: double
total_amount: double
congestion_surcharge: double
airport_fee: double
cbd_congestion_fee: double


In [7]:
import pyarrow as pa

arrow_schema = pa.schema([
    ('vendor_id', pa.int32()),
    ('tpep_pickup_datetime', pa.timestamp('us')),
    ('tpep_dropoff_datetime', pa.timestamp('us')),
    ('passenger_count', pa.int64()),
    ('trip_distance', pa.float64()),
    ('ratecode_id', pa.int64()),
    ('store_and_fwd_flag', pa.string()),          # large_string -> pa.string()
    ('pu_location_id', pa.int32()),
    ('do_location_id', pa.int32()),
    ('payment_type', pa.int64()),
    ('fare_amount', pa.float64()),
    ('extra', pa.float64()),
    ('mta_tax', pa.float64()),
    ('tip_amount', pa.float64()),
    ('tolls_amount', pa.float64()),
    ('improvement_surcharge', pa.float64()),
    ('total_amount', pa.float64()),
    ('congestion_surcharge', pa.float64()),
    ('airport_fee', pa.float64()),
    ('cbd_congestion_fee', pa.float64())
])


In [8]:
from pyiceberg.catalog import load_catalog

catalog = load_catalog(
    "nessie",
    **{
        "uri": "http://nessie:19120/iceberg/main",   # nombre del servicio en compose        # bucket en MinIO
    }
)


In [9]:
namespaces = catalog.list_namespaces()
print("Namespaces:", namespaces)

Namespaces: []


In [10]:
catalog.create_namespace("project")

In [11]:
namespaces = catalog.list_namespaces()
print("Namespaces:", namespaces)

Namespaces: [('project',)]


In [12]:
import boto3
s3 = boto3.resource(
    's3',
    endpoint_url="http://minio:9000",  # tu endpoint de MinIO
    aws_access_key_id='admin',             # tu access key
    aws_secret_access_key='password'       # tu secret key
)

In [13]:
# -----------------------
# Crear tabla Iceberg apuntando a MinIO
# -----------------------
catalog.create_table(
    "project.projecticeberg2",
    schema=arrow_schema,
    location="s3://projecticeberg/projecticeberg",  # Ruta en MinIO
    properties={
        "fs.s3a.endpoint": "http://minio:9000",
        "fs.s3a.access.key": "admin",
        "fs.s3a.secret.key": "password",
        "fs.s3a.path.style.access": "true"
    }
)

projecticeberg2(
  1: vendor_id: optional int,
  2: tpep_pickup_datetime: optional timestamp,
  3: tpep_dropoff_datetime: optional timestamp,
  4: passenger_count: optional long,
  5: trip_distance: optional double,
  6: ratecode_id: optional long,
  7: store_and_fwd_flag: optional string,
  8: pu_location_id: optional int,
  9: do_location_id: optional int,
  10: payment_type: optional long,
  11: fare_amount: optional double,
  12: extra: optional double,
  13: mta_tax: optional double,
  14: tip_amount: optional double,
  15: tolls_amount: optional double,
  16: improvement_surcharge: optional double,
  17: total_amount: optional double,
  18: congestion_surcharge: optional double,
  19: airport_fee: optional double,
  20: cbd_congestion_fee: optional double
),
partition by: [],
sort order: [],
snapshot: null

In [ ]:
# catalog.create_table("project.project_iceberg", schema =arrow_schema)

In [15]:
#cargar tabla iceberg y hacemos el append
project= catalog.load_table("project.projecticeberg2")

In [ ]:
project.append(table)

CONSULTAR DATOS

In [ ]:
#Escaneamos la tabla y la traemos a arrow

result_arrow = project.scan().to_arrow()

resultdf = result_arrow.to_pandas()

print("Datos leidos desde Iceberg: ")
print("Total de filas: ", len(resultdf))

In [ ]:
resultdf.head()